In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os
from langchain_ollama import ChatOllama

model_name="granite4:1b"
model_url=os.getenv('OLLAMA_HOST')

model = ChatOllama(
    model=model_name,
    api_base=model_url
)

In [3]:
from langchain.agents import AgentState

class CustomState(AgentState):
    favourite_colour: str

## Write to state

In [4]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def update_favourite_colour(favourite_colour: str, runtime: ToolRuntime) -> Command:
    """Update the favourite colour of the user in the state once they've revealed it."""
    return Command(update={
        "favourite_colour": favourite_colour, 
        "messages": [ToolMessage("Successfully updated favourite colour", tool_call_id=runtime.tool_call_id)]}
        )

In [5]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model=model,
    tools=[update_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [6]:
from langchain.messages import HumanMessage

response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

In [7]:
from pprint import pprint

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='e4e706a1-6224-4321-a2f0-00da72e1f2a9'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-25T23:57:49.5241827Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3971045500, 'load_duration': 2519728300, 'prompt_eval_count': 185, 'prompt_eval_duration': 288004700, 'eval_count': 24, 'eval_duration': 1099371100, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019b57f2-6ae5-79a2-8dd7-92cbb43085d0-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favourite_colour': 'green'}, 'id': '5c9a35f5-f519-4863-8db2-5798f913a230', 'type': 'tool_call'}], usage_metadata={'input_tokens': 185, 'output_tokens': 24, 'total_tokens': 209}),
              ToolMessage(content='Successfully updated favourite colour', name='update_favourite_

In [8]:
response = agent.invoke(
    { 
        "messages": [HumanMessage(content="Hello, how are you?")],
        "favourite_colour": "green"
    },
    {"configurable": {"thread_id": "10"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='a3c615c1-e925-4dd5-91c0-94e602acc450'),
              AIMessage(content="I'm just an AI assistant, but I'm here and ready to help! How can I assist you today?", additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-25T23:58:00.4168997Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1635099300, 'load_duration': 76007900, 'prompt_eval_count': 186, 'prompt_eval_duration': 392330800, 'eval_count': 24, 'eval_duration': 1075343200, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019b57f2-9e9b-7b00-a290-691bacc92f24-0', usage_metadata={'input_tokens': 186, 'output_tokens': 24, 'total_tokens': 210})]}


## Read state

In [10]:
@tool
def read_favourite_colour(runtime: ToolRuntime) -> str:
    """Read the favourite colour of the user from the state."""
    try:
        return runtime.state["favourite_colour"]
    except KeyError:
        return "No favourite colour found in state"

agent = create_agent(
    model=model,
    tools=[update_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [11]:
response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='7ec40b0a-0d47-47fb-ac03-96db175703db'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-25T23:58:42.3461071Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1513111800, 'load_duration': 80549400, 'prompt_eval_count': 185, 'prompt_eval_duration': 255907300, 'eval_count': 24, 'eval_duration': 1077701900, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019b57f3-42ce-7982-ba61-b1a13648e644-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favourite_colour': 'green'}, 'id': '83cc6600-ddc1-429c-9f78-275754cfd09c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 185, 'output_tokens': 24, 'total_tokens': 209}),
              ToolMessage(content='Successfully updated favourite colour', name='update_favourite_co

In [12]:
response = agent.invoke(
    { "messages": [HumanMessage(content="What's my favourite colour?")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='7ec40b0a-0d47-47fb-ac03-96db175703db'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-25T23:58:42.3461071Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1513111800, 'load_duration': 80549400, 'prompt_eval_count': 185, 'prompt_eval_duration': 255907300, 'eval_count': 24, 'eval_duration': 1077701900, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019b57f3-42ce-7982-ba61-b1a13648e644-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favourite_colour': 'green'}, 'id': '83cc6600-ddc1-429c-9f78-275754cfd09c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 185, 'output_tokens': 24, 'total_tokens': 209}),
              ToolMessage(content='Successfully updated favourite colour', name='update_favourite_co